# Settings

## import libraries

In [1]:
import keyring
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

## connect databases

In [2]:
# test for connecting databases
user = 'root'
pw = keyring.get_password('macmini_db', user)
host = '192.168.219.106'
engine = create_engine(f'mysql+pymysql://{user}:{pw}@{host}:3306/stock')
price = pd.read_sql('SELECT * FROM sample_etf;', con=engine)
engine.dispose()

In [3]:
price

,Date,SPY,IEV,EWJ,EEM,TLT,IEF,IYR,RWX,GLD,DBC
0,1993-01-29,24.941397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1993-02-01,25.118792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1993-02-02,25.172007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1993-02-03,25.438086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1993-02-04,25.544548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
7760,2023-11-22,455.019989,50.160000,62.490002,39.520000,90.870003,92.750000,82.250000,24.920000,184.559998,24.180000
7761,2023-11-24,455.299988,50.669998,62.610001,39.540001,89.800003,92.269997,82.550003,25.049999,185.520004,24.020000
7762,2023-11-27,454.480011,50.500000,62.560001,39.380001,91.300003,92.970001,82.849998,25.049999,186.770004,23.860001
7763,2023-11-28,454.929993,50.490002,62.549999,39.709999,91.480003,93.379997,83.339996,25.120001,189.259995,24.150000


# Update ticker_kr Database

## Get recent business day of Korea

In [4]:
# import libraries

import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one('div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)

  |  2024.03.07


In [5]:
# regex
import re

biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)
biz_day

'20240307'

## Get sector category info

### crawling KOSPI from KRX

In [6]:
# get OTP

import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_stk = {
    'mktId' : 'STK',        # STK는 코스피
    'trdDd' : biz_day,
    'money' : '1',
    'csvxls_isNo' : 'false',
    'name' : 'fileDown',
    'url' : 'dbms/MDC/STAT/standard/MDCSTAT03901'
}

# add a referrer in the header
# we can get OTP from the first url, when sending this to the second url without a referrer, web site recognizes this request as one from a bot.
headers = {'Referer':  'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
# send queries by post() funciton, get data and select only text info.
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text

print(otp_stk)

fjhTvrj2mv1NmfNJZx4OIJ7nWZ+Bl166nwvyorXiFn0RtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15eWaJ70ZJUNInhRFusVfcL2otBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4m8vFLhSKmM794gFu+ThsO31lY4woqehX8j6OlXFDcfHdV4NbYo4+D2Rwcfj24VnU3Zpq3ik/Dyw3FdyOXhJkBI=


In [7]:
# download data after sending OTP
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_stk = rq.post(down_url, {'code': otp_stk}, headers=headers)
# 1. convert recieved data's content part into binary stream by ByteIO() and read by read_csv() function.
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR')

sector_stk.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,095570,AJ네트웍스,KOSPI,서비스업,4510,-50,-1.10,204089943090
1,006840,AK홀딩스,KOSPI,기타금융,15970,30,0.19,211563549170
2,027410,BGF,KOSPI,기타금융,3765,-55,-1.44,360373718115
3,282330,BGF리테일,KOSPI,유통업,122500,-7900,-6.06,2117278485000
4,138930,BNK금융지주,KOSPI,기타금융,7900,210,2.73,2544498660200


### crawling KOSDAQ from KRX

In [8]:
# download KOSDAQ data
gen_otp_ksq = {
    'mktId': 'KSQ',      # 코스닥 코드
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}

otp_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text

down_sector_ksq = rq.post(down_url, {'code': otp_ksq}, headers=headers)
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding='EUC-KR')

sector_ksq.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,2790,0,0.00,135417231180
1,054620,APS,KOSDAQ,금융,6850,80,1.18,139700413850
2,265520,AP시스템,KOSDAQ,반도체,21750,400,1.87,332370906750
3,211270,AP위성,KOSDAQ,통신장비,15560,-500,-3.11,234680650240
4,139050,BF랩스,KOSDAQ,컴퓨터서비스,7310,-180,-2.40,62808331410


In [9]:
# combine KOSPI and KOSDAQ data
krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True)
# delete blank in the company name only if there is.
krx_sector['종목명'] = krx_sector['종목명'].str.strip()
# add 'base_data' column
krx_sector['base_date'] = biz_day

krx_sector.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,base_date
0,095570,AJ네트웍스,KOSPI,서비스업,4510,-50,-1.10,204089943090,20240307
1,006840,AK홀딩스,KOSPI,기타금융,15970,30,0.19,211563549170,20240307
2,027410,BGF,KOSPI,기타금융,3765,-55,-1.44,360373718115,20240307
3,282330,BGF리테일,KOSPI,유통업,122500,-7900,-6.06,2117278485000,20240307
4,138930,BNK금융지주,KOSPI,기타금융,7900,210,2.73,2544498660200,20240307


## Class for updatting `ticker_kr` database

In [10]:
import requests as rq
from bs4 import BeautifulSoup
import re
from io import BytesIO
import pandas as pd
import numpy as np
import keyring
import pymysql

class UpdateTickerKrDB:
    
    def __init__(self):
        self.biz_day = self.get_recent_biz_day()
        self.gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
        self.mktID = {
            'KOSPI' : 'STK',    # KOSPI ID
            'KOSDAQ' : 'KSQ'    # KOSDAQ ID
        }
        self.biz_day = self.get_recent_biz_day()
        
        # add a referrer in the header
        # we can get OTP from the first url, when sending this to the second url without a referrer, web site recognizes this request as one from a bot.
        self.headers = {'Referer':  'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
        
    def get_recent_biz_day(self):
        
        # get recent biz day from Naver finance
        url = 'https://finance.naver.com/sise/sise_deposit.nhn'
        data = rq.get(url)
        data_html = BeautifulSoup(data.content)
        parse_day = data_html.select_one('div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text
        
        # regex for converting date into 'yyyymmdd' format
        biz_day = re.findall('[0-9]+', parse_day)
        biz_day = ''.join(biz_day)
        
        return biz_day
    
    def gen_otp_krx(self, market='KOSPI'):
        
        if market == 'Ind':
            # generate OTP for individual stock info
            gen_otp = {
                'searchType': '1',
                'mktId': 'ALL',
                'trdDd': self.biz_day,
                'csvxls_isNo': 'false',
                'name': 'fileDown',
                'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
            }
        else:
            # generate OTP for KOSPI/KOSDAQ market info
            gen_otp = {
                'mktId' : self.mktID[market],        # STK는 코스피
                'trdDd' : self.biz_day,
                'money' : '1',
                'csvxls_isNo' : 'false',
                'name' : 'fileDown',
                'url' : 'dbms/MDC/STAT/standard/MDCSTAT03901'
            }
        
        # send queries by post() funciton, get data and select only text info.
        otp_stk = rq.post(self.gen_otp_url, gen_otp, headers=self.headers).text

        print(f"generating OTP mktID: {gen_otp['mktId']}")
        
        return otp_stk
    
    def download_data_krx(self, market='KOSPI'):
        
        # generate OTP
        otp = self.gen_otp_krx(market)
        down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
        down_sector = rq.post(down_url, {'code': otp}, headers=self.headers)
        
        sector = pd.read_csv(BytesIO(down_sector.content), encoding='EUC-KR')
        
        return sector
    
    def get_combined_data_krx(self):
        
        # download KOSPI, KOSDAQ and individual stock info
        sector_stk = self.download_data_krx(market='KOSPI')
        sector_ksq = self.download_data_krx(market='KOSDAQ')
        krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True)
        
        # delete blank in the company_name
        krx_sector['종목명'] = krx_sector['종목명'].str.strip()
        
        # add 'date' column for recent business day
        krx_sector['date'] = self.biz_day
        
        # get individual stock data from KRX
        krx_ind = self.download_data_krx(market='Ind')
        krx_ind['종목명'] = krx_ind['종목명'].str.strip()
        krx_ind['data'] = self.biz_day
        
        # get difference list between krx_sector and krx_ind
        diff = list(set(krx_sector['종목명']).symmetric_difference(set(krx_ind['종목명'])))
        
        # diff codes are not normal, so we just merge two data bases
        kor_ticker = pd.merge(
            krx_sector,
            krx_ind,
            on = krx_sector.columns.intersection(
                krx_ind.columns
            ).to_list(), how='outer'
        )
        
        # distinguish general stock from SPAC, preferred stock, REITs, other stocks
        kor_ticker['category'] = np.where(kor_ticker['종목명'].str.contains('스팩|제[0-9]+호'), '스팩',
                                          np.where(kor_ticker['종목코드'].str[-1:] != '0', '우선주',
                                                   np.where(kor_ticker['종목명'].str.endswith('리츠'), '리츠',
                                                            np.where(kor_ticker['종목명'].isin(diff), '기타', '보통주'))))
        
        kor_ticker = kor_ticker.reset_index(drop=True)
        kor_ticker.columns = kor_ticker.columns.str.replace(' ', '')    # delete blank from column names
        kor_ticker = kor_ticker[['종목코드', '종목명', '시장구분', '종가',
                                 '시가총액', 'date', 'EPS', '선행EPS', 'BPS', '주당배당금', 'category']]
        kor_ticker['date'] = self.biz_day
        
        # rename columns name into english
        kor_ticker.rename(columns={
            '종목코드': 'company_code',
            '종목명': 'company',
            '시장구분': 'market',
            '종가': 'close',
            '시가총액': 'market_cap', 
            'EPS': 'eps', 
            '선행EPS': 'forward_eps', 
            'BPS': 'bps', 
            '주당배당금': 'dividend'
        }, inplace=True)
        
        return kor_ticker
    
    def update_db_ticker_kr(self, kor_ticker):
        
        # replace NaN into None
        kor_ticker = kor_ticker.replace({np.nan: None})
        
        # database info
        user = 'root'
        pw = keyring.get_password('macmini_db', user)
        host = '192.168.219.106'
        
        con = pymysql.connect(
            user=user,
            passwd=pw,
            host=host,
            db='stock',
            charset='utf8'
        )
        
        mycursor = con.cursor()
        query = f"""
        INSERT INTO ticker_kr (company_code, company, market, close, market_cap, date, eps, forward_eps, bps, dividend, category)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
        company=VALUES(company), market=VALUES(market), close=VALUES(close), market_cap=VALUES(market_cap), eps=VALUES(eps),
        forward_eps=VALUES(forward_eps), bps=VALUES(bps), dividend=VALUES(dividend), category=VALUES(category);
        """
        
        args = kor_ticker.values.tolist()
        
        mycursor.executemany(query, args)
        con.commit()
        
        con.close()
        


In [11]:
# get combined data from krx
a = UpdateTickerKrDB()
data = a.get_combined_data_krx()
data

generating OTP mktID: STK
generating OTP mktID: KSQ
generating OTP mktID: ALL


,company_code,company,market,close,market_cap,date,eps,forward_eps,bps,dividend,category
0,095570,AJ네트웍스,KOSPI,4510,204089943090,20240307,201.0,685.0,8076.0,270.0,보통주
1,006840,AK홀딩스,KOSPI,15970,211563549170,20240307,NaN,NaN,41948.0,200.0,보통주
2,027410,BGF,KOSPI,3765,360373718115,20240307,247.0,663.0,16528.0,110.0,보통주
3,282330,BGF리테일,KOSPI,122500,2117278485000,20240307,11203.0,12877.0,55724.0,4100.0,보통주
4,138930,BNK금융지주,KOSPI,7900,2544498660200,20240307,2404.0,2453.0,30468.0,625.0,보통주
...,...,...,...,...,...,...,...,...,...,...,...
2665,024060,흥구석유,KOSDAQ,9830,147450000000,20240307,183.0,NaN,5508.0,150.0,보통주
2666,010240,흥국,KOSDAQ,5640,69500005440,20240307,740.0,NaN,7971.0,220.0,보통주
2667,189980,흥국에프엔비,KOSDAQ,2220,89105975940,20240307,309.0,NaN,2295.0,40.0,보통주
2668,037440,희림,KOSDAQ,6680,93002133000,20240307,567.0,NaN,5186.0,150.0,보통주


In [12]:
# update ticker_kr database
a.update_db_ticker_kr(data)

# Update sector kr database

## Data exploration

In [13]:
def get_recent_biz_day():
        
        # get recent biz day from Naver finance
        url = 'https://finance.naver.com/sise/sise_deposit.nhn'
        data = rq.get(url)
        data_html = BeautifulSoup(data.content)
        parse_day = data_html.select_one('div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text
        
        # regex
        biz_day = re.findall('[0-9]+', parse_day)
        biz_day = ''.join(biz_day)
        
        return biz_day

In [14]:
import json
import requests as rq
import pandas as pd

biz_day = get_recent_biz_day()

url = f'''http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={biz_day}&sec_cd=G10'''  
data = rq.get(url).json()

data

{'info': {'TRD_DT': '/Date(1709737200000)/',
  'MKT_VAL': 21218229,
  'TRD_AMT': 192014,
  'CNT': 33},
 'list': [{'IDX_CD': 'G10',
   'IDX_NM_KOR': 'WICS 에너지',
   'ALL_MKT_VAL': 21218229,
   'CMP_CD': '096770',
   'CMP_KOR': 'SK이노베이션',
   'MKT_VAL': 6369097,
   'WGT': 30.02,
   'S_WGT': 30.02,
   'CAL_WGT': 1.0,
   'SEC_CD': 'G10',
   'SEC_NM_KOR': '에너지',
   'SEQ': 1,
   'TOP60': 4,
   'APT_SHR_CNT': 53611930},
  {'IDX_CD': 'G10',
   'IDX_NM_KOR': 'WICS 에너지',
   'ALL_MKT_VAL': 21218229,
   'CMP_CD': '267250',
   'CMP_KOR': 'HD현대',
   'MKT_VAL': 3087682,
   'WGT': 14.55,
   'S_WGT': 44.57,
   'CAL_WGT': 1.0,
   'SEC_CD': 'G10',
   'SEC_NM_KOR': '에너지',
   'SEQ': 2,
   'TOP60': 4,
   'APT_SHR_CNT': 44236128},
  {'IDX_CD': 'G10',
   'IDX_NM_KOR': 'WICS 에너지',
   'ALL_MKT_VAL': 21218229,
   'CMP_CD': '010950',
   'CMP_KOR': 'S-Oil',
   'MKT_VAL': 3049192,
   'WGT': 14.37,
   'S_WGT': 58.94,
   'CAL_WGT': 1.0,
   'SEC_CD': 'G10',
   'SEC_NM_KOR': '에너지',
   'SEQ': 3,
   'TOP60': 4,
   'APT_SHR

In [15]:
data.keys()

dict_keys(['info', 'list', 'sector', 'size'])

In [16]:
# stocks are in the 'list' and sector codes are in the 'sector'
data['list'][0]

{'IDX_CD': 'G10',
 'IDX_NM_KOR': 'WICS 에너지',
 'ALL_MKT_VAL': 21218229,
 'CMP_CD': '096770',
 'CMP_KOR': 'SK이노베이션',
 'MKT_VAL': 6369097,
 'WGT': 30.02,
 'S_WGT': 30.02,
 'CAL_WGT': 1.0,
 'SEC_CD': 'G10',
 'SEC_NM_KOR': '에너지',
 'SEQ': 1,
 'TOP60': 4,
 'APT_SHR_CNT': 53611930}

In [17]:
data['sector']

[{'SEC_CD': 'G25', 'SEC_NM_KOR': '경기관련소비재', 'SEC_RATE': 10.4, 'IDX_RATE': 0},
 {'SEC_CD': 'G35', 'SEC_NM_KOR': '건강관리', 'SEC_RATE': 9.38, 'IDX_RATE': 0},
 {'SEC_CD': 'G50', 'SEC_NM_KOR': '커뮤니케이션서비스', 'SEC_RATE': 5.77, 'IDX_RATE': 0},
 {'SEC_CD': 'G40', 'SEC_NM_KOR': '금융', 'SEC_RATE': 9.23, 'IDX_RATE': 0},
 {'SEC_CD': 'G10', 'SEC_NM_KOR': '에너지', 'SEC_RATE': 1.66, 'IDX_RATE': 100.0},
 {'SEC_CD': 'G20', 'SEC_NM_KOR': '산업재', 'SEC_RATE': 12.5, 'IDX_RATE': 0},
 {'SEC_CD': 'G55', 'SEC_NM_KOR': '유틸리티', 'SEC_RATE': 1.09, 'IDX_RATE': 0},
 {'SEC_CD': 'G30', 'SEC_NM_KOR': '필수소비재', 'SEC_RATE': 1.99, 'IDX_RATE': 0},
 {'SEC_CD': 'G15', 'SEC_NM_KOR': '소재', 'SEC_RATE': 8.64, 'IDX_RATE': 0},
 {'SEC_CD': 'G45', 'SEC_NM_KOR': 'IT', 'SEC_RATE': 39.35, 'IDX_RATE': 0}]

In [18]:
sector_codes = []
for i in range(len(data['sector'])):
    sector_code = data['sector'][i]['SEC_CD']
    sector_codes.append(sector_code)
    
print(sector_codes)

['G25', 'G35', 'G50', 'G40', 'G10', 'G20', 'G55', 'G30', 'G15', 'G45']


In [19]:
sector_names = []
for i in range(len(data['sector'])):
    sector_name = data['sector'][i]['SEC_NM_KOR']
    sector_names.append(sector_name)
    
print(sector_names)

['경기관련소비재', '건강관리', '커뮤니케이션서비스', '금융', '에너지', '산업재', '유틸리티', '필수소비재', '소재', 'IT']


In [20]:
# transform list into dataframe
data_pd = pd.json_normalize(data['list'])
data_pd.head()

,IDX_CD,IDX_NM_KOR,ALL_MKT_VAL,CMP_CD,CMP_KOR,MKT_VAL,WGT,S_WGT,CAL_WGT,SEC_CD,SEC_NM_KOR,SEQ,TOP60,APT_SHR_CNT
0,G10,WICS 에너지,21218229,096770,SK이노베이션,6369097,30.02,30.02,1.0,G10,에너지,1,4,53611930
1,G10,WICS 에너지,21218229,267250,HD현대,3087682,14.55,44.57,1.0,G10,에너지,2,4,44236128
2,G10,WICS 에너지,21218229,010950,S-Oil,3049192,14.37,58.94,1.0,G10,에너지,3,4,41655633
3,G10,WICS 에너지,21218229,009830,한화솔루션,2885990,13.60,72.54,1.0,G10,에너지,4,4,108292298
4,G10,WICS 에너지,21218229,078930,GS,2405626,11.34,83.88,1.0,G10,에너지,5,4,49245150


## Class for updating `sector_kr` database

In [21]:
import json
import requests as rq
import pandas as pd
import keyring
import pymysql
import time
from tqdm import tqdm

class UpdateSectorKrDB:
    
    def __init__(self):
        self.biz_day = self.get_recent_biz_day()
        self.url = f'http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={self.biz_day}&sec_cd=G10'        
    
    def get_recent_biz_day(self):
        
        # get recent biz day from Naver finance
        url = 'https://finance.naver.com/sise/sise_deposit.nhn'
        data = rq.get(url)
        data_html = BeautifulSoup(data.content)
        parse_day = data_html.select_one('div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text
        
        # regex
        biz_day = re.findall('[0-9]+', parse_day)
        biz_day = ''.join(biz_day)
        
        return biz_day
    
    def get_sector_codes(self):
        
        data = rq.get(self.url).json()
        sector_codes = []
        for i in range(len(data['sector'])):
            sector_code = data['sector'][i]['SEC_CD']
            sector_codes.append(sector_code)
        
        return sector_codes
    
    def get_data_from_wics(self):
        sector_codes = self.get_sector_codes()
        
        data_sector = []
        
        for i in tqdm(sector_codes):
            url = f'''http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={self.biz_day}&sec_cd={i}'''
            data = rq.get(url).json()
            data_pd = pd.json_normalize(data['list'])
            
            data_sector.append(data_pd)
            
            time.sleep(2)
            
        # turn data into dataframe
        kor_sector = pd.concat(data_sector, axis=0)
        kor_sector = kor_sector[['IDX_CD', 'CMP_CD', 'CMP_KOR', 'SEC_NM_KOR']]
        kor_sector['date'] = self.biz_day
        kor_sector['date'] = pd.to_datetime(kor_sector['date'])
        
        # rename columns
        kor_sector.rename(columns={
            'IDX_CD': 'index_code', 
            'CMP_CD': 'company_code', 
            'CMP_KOR': 'company', 
            'SEC_NM_KOR': 'sec_nm_kor'
        }, inplace=True)
            
        return kor_sector

    def update_db_sector_kr(self, kor_sector):
        
        # replace NaN into None
        kor_sector = kor_sector.replace({np.nan: None})
        
        # database info
        user = 'root'
        pw = keyring.get_password('macmini_db', user)
        host = '192.168.219.106'
        
        con = pymysql.connect(
            user=user,
            passwd=pw,
            host=host,
            db='stock',
            charset='utf8'
        )
        
        mycursor = con.cursor()
        query = f"""
        INSERT INTO sector_kr (index_code, company_code, company, sec_nm_kr, date)
        VALUES (%s,%s,%s,%s,%s)
        ON DUPLICATE KEY UPDATE
        index_code=VALUES(index_code), company_code=VALUES(company_code), company=VALUES(company), sec_nm_kr=VALUES(sec_nm_kr)
        """
        
        args = kor_sector.values.tolist()
        
        mycursor.executemany(query, args)
        con.commit()
        
        con.close()      

In [22]:
b = UpdateSectorKrDB()
data = b.get_data_from_wics()
data

100%|██████████| 10/10 [00:24<00:00,  2.43s/it]


,index_code,company_code,company,sec_nm_kor,date
0,G25,005380,현대차,경기관련소비재,2024-03-07
1,G25,000270,기아,경기관련소비재,2024-03-07
2,G25,012330,현대모비스,경기관련소비재,2024-03-07
3,G25,161390,한국타이어앤테크놀로지,경기관련소비재,2024-03-07
4,G25,090430,아모레퍼시픽,경기관련소비재,2024-03-07
...,...,...,...,...,...
670,G45,067770,세진티에스,IT,2024-03-07
671,G45,424760,벨로크,IT,2024-03-07
672,G45,290560,신시웨이,IT,2024-03-07
673,G45,372800,아이티아이즈,IT,2024-03-07


In [23]:
b.update_db_sector_kr(data)

# Update `price_kr` database

## data exploration

In [24]:
# crawling stock price
from dateutil.relativedelta import relativedelta
import requests as rq
from io import BytesIO
from datetime import date

i = 0
ticker = '000020'
fr = (date.today() + relativedelta(years=-40)).strftime("%Y%m%d")
to = (date.today().strftime("%Y%m%d"))

url = f'https://fchart.stock.naver.com/siseJson.nhn?symbol={ticker}&requestType=1&startTime={fr}&endTime={to}&timeframe=day'

data = rq.get(url).content
data_price = pd.read_csv(BytesIO(data))
data_price

,[['날짜','시가','고가','저가','종가','거래량','외국인소진율'],Unnamed: 7
0,"[""19900103""",22500.0,22500.0,22500.0,22500.0,10.0,],NaN
1,"[""19900104""",22200.0,23500.0,22200.0,23000.0,7230.0,],NaN
2,"[""19900105""",22500.0,22800.0,22500.0,22500.0,1280.0,],NaN
3,"[""19900106""",22200.0,22200.0,22200.0,22200.0,1140.0,],NaN
4,"[""19900108""",22200.0,22700.0,22200.0,22700.0,2370.0,],NaN
...,...,...,...,...,...,...,...,...
8842,"[""20240305""",9690.0,9820.0,9690.0,9720.0,35992.0,5.99],NaN
8843,"[""20240306""",9700.0,9810.0,9640.0,9720.0,95288.0,6.11],NaN
8844,"[""20240307""",9730.0,9740.0,9540.0,9550.0,99042.0,6.05],NaN
8845,"[""20240308""",9570.0,9620.0,9490.0,9530.0,53071.0,6.05],NaN


In [25]:
# clean data
import re

price = data_price.iloc[:, 0:6]
price.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
price = price.dropna()
price['date'] = price['date'].str.extract('(\d+)')
price['date'] = pd.to_datetime(price['date'])
price['company_code'] = ticker

price.head()

,date,open,high,low,close,volume,company_code
0,1990-01-03,22500.0,22500.0,22500.0,22500.0,10.0,000020
1,1990-01-04,22200.0,23500.0,22200.0,23000.0,7230.0,000020
2,1990-01-05,22500.0,22800.0,22500.0,22500.0,1280.0,000020
3,1990-01-06,22200.0,22200.0,22200.0,22200.0,1140.0,000020
4,1990-01-08,22200.0,22700.0,22200.0,22700.0,2370.0,000020


## Class for updating `price_kr` database

In [26]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import keyring
from dateutil.relativedelta import relativedelta
import requests as rq
from io import BytesIO
from datetime import date
import re
import time
from tqdm import tqdm

class UpdatePriceKrDB:
    
    def __init__(self):
        
        self.ticker_list = self.read_ticker_list()
        
    def read_ticker_list(self):
        # database info
        user = 'root'
        pw = keyring.get_password('macmini_db', user)
        host = '192.168.219.106'
        port = 3306
        db = 'stock'
        
        # connect database
        engine = create_engine(f'mysql+pymysql://{user}:{pw}@{host}:{port}/{db}')
        query = """ 
        SELECT * FROM ticker_kr
        WHERE date = (SELECT MAX(date) FROM ticker_kr)
            AND category = '보통주';
        """
        
        # get ticker list from ticker_kr database
        ticker_list = pd.read_sql(query, con=engine)
        engine.dispose()
        
        return ticker_list
    
    def update_price_db_naver(self):
        
        # database info
        user = 'root'
        pw = keyring.get_password('macmini_db', user)
        host = '192.168.219.106'
        port = 3306
        db = 'stock'
        
        # connect database
        engine = create_engine(f'mysql+pymysql://{user}:{pw}@{host}:{port}/{db}')
        con = pymysql.connect(
            user=user,
            passwd=pw,
            host=host,
            db=db,
            charset='utf8'
        )
        mycursor = con.cursor()
        
        # get ticker list
        tikcer_list = self.read_ticker_list()

        query = """ 
        INSERT INTO price_kr (date, open, high, low, close, volume, company_code)
        VALUES (%s,%s,%s,%s,%s,%s,%s)
        ON DUPLICATE KEY UPDATE
        open=VALUES(open), high=VALUES(high), low=VALUES(low), close=VALUES(close), volume=VALUES(volume);
        """
        
        # list for errors
        error_list = []
        
        # download for all price data (for 50 years)
        for i in tqdm(range(0, len(self.ticker_list))):
            
            # pick a ticker
            ticker = self.ticker_list['company_code'][i]
            
            # start date and end date
            fr = (date.today() + relativedelta(years=-50)).strftime('%Y%m%d')
            to = (date.today()).strftime('%Y%m%d')
            
            # error occurs, skip and do next loop
            try:
                # url : crawling data from Naver
                url = f'''https://fchart.stock.naver.com/siseJson.nhn?symbol={ticker}&requestType=1&startTime={fr}&endTime={to}&timeframe=day'''
                
                # download data
                data = rq.get(url).content
                data_price = pd.read_csv(BytesIO(data))
                
                # data cleaning
                price = data_price.iloc[:, 0:6]
                price.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
                price = price.dropna()
                price['date'] = price['date'].str.extract('(\d+)')      # regex : extract only numbers from columns
                price['date'] = pd.to_datetime(price['date'])
                price['company_code'] = ticker
                
                # insert db
                args = price.values.tolist()
                mycursor.executemany(query, args)
                con.commit()
            
            except:
                print(ticker)
                error_list.append(ticker)
                
            # time sleep
            time.sleep(2)
            
        # close db connection
        engine.dispose()
        con.close()
                                                             

In [27]:
a = UpdatePriceKrDB()
a.update_price_db_naver()

100%|██████████| 2418/2418 [1:50:18<00:00,  2.74s/it]  


# Update kr financial statements 

## Crawl financial statements

### Data exploration

In [25]:
# get data from Samsung

from sqlalchemy import create_engine
import pandas as pd
import keyring

# connect db
user = 'root'
pw = keyring.get_password('macmini_db', user)
host = '192.168.219.106'
port = 3306
db = 'stock'

engine = create_engine(f'mysql+pymysql://{user}:{pw}@{host}:{port}/{db}')
query = """ 
SELECT * FROM ticker_kr
WHERE date = {SELECT MAX(date) FROM ticker_kr}
    AND category = '보통주';
"""
engine.dispose()

i = 0
ticker = '005930'

url = f"http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{ticker}"
data = pd.read_html(url, displayed_only=False)

# data
[item.head(3) for item in data]

[  IFRS(연결)    2020/12    2021/12    2022/12    2023/12       전년동기 전년동기(%)
 0      매출액  2368070.0  2796048.0  3022314.0  2589355.0  3022314.0   -14.3
 1     매출원가  1444883.0  1664113.0  1900418.0  1803886.0  1900418.0    -5.1
 2    매출총이익   923187.0  1131935.0  1121896.0   785469.0  1121896.0   -30.0,
   IFRS(연결)   2023/03   2023/06   2023/09   2023/12      전년동기 전년동기(%)
 0      매출액  637454.0  600055.0  674047.0  677799.0  704646.0    -3.8
 1     매출원가  460071.0  416472.0  466187.0  461156.0  486277.0    -5.2
 2    매출총이익  177383.0  183583.0  207859.0  216644.0  218368.0    -0.8,
              IFRS(연결)    2020/12    2021/12    2022/12    2023/12
 0                  자산  3782357.0  4266212.0  4484245.0  4559060.0
 1  유동자산계산에 참여한 계정 펼치기  1982156.0  2181632.0  2184706.0  1959366.0
 2                재고자산   320431.0   413844.0   521879.0   516259.0,
              IFRS(연결)    2023/03    2023/06    2023/09    2023/12
 0                  자산  4540918.0  4480006.0  4544664.0  4559060.0
 1  유동자산계산에 참여한

In [26]:
print(
    data[0].columns.tolist(), '\n',
    data[2].columns.tolist(), '\n',
    data[4].columns.to_list()
)

['IFRS(연결)', '2020/12', '2021/12', '2022/12', '2023/12', '전년동기', '전년동기(%)'] 
 ['IFRS(연결)', '2020/12', '2021/12', '2022/12', '2023/12'] 
 ['IFRS(연결)', '2020/12', '2021/12', '2022/12', '2023/12']


In [27]:
data_fs_y = pd.concat(
    [data[0].iloc[:, ~data[0].columns.str.contains('전년동기')], data[2], data[4]]
)
data_fs_y = data_fs_y.rename(columns={data_fs_y.columns[0]: "account"})
data_fs_y.head()

,account,2020/12,2021/12,2022/12,2023/12
0,매출액,2368070.0,2796048.0,3022314.0,2589355.0
1,매출원가,1444883.0,1664113.0,1900418.0,1803886.0
2,매출총이익,923187.0,1131935.0,1121896.0,785469.0
3,판매비와관리비계산에 참여한 계정 펼치기,563248.0,615596.0,688130.0,719799.0
4,인건비,70429.0,75568.0,80937.0,86239.0


In [28]:
# 결산월 추출
import requests as rq
from bs4 import BeautifulSoup
import re

page_data = rq.get(url)
page_data_html = BeautifulSoup(page_data.content, 'html.parser')

fiscal_data = page_data_html.select('div.corp_group1 > h2')
fiscal_data_text = fiscal_data[1].text
fiscal_data_text = re.findall('[0-9]+', fiscal_data_text)

print(fiscal_data_text)

['12']


In [29]:
# 재무제표에 해당하는 열만 선택
data_fs_y = data_fs_y.loc[:, (data_fs_y.columns == 'account') |
                          (data_fs_y.columns.str[-2:].isin(fiscal_data_text))]
data_fs_y.head()

,account,2020/12,2021/12,2022/12,2023/12
0,매출액,2368070.0,2796048.0,3022314.0,2589355.0
1,매출원가,1444883.0,1664113.0,1900418.0,1803886.0
2,매출총이익,923187.0,1131935.0,1121896.0,785469.0
3,판매비와관리비계산에 참여한 계정 펼치기,563248.0,615596.0,688130.0,719799.0
4,인건비,70429.0,75568.0,80937.0,86239.0
